# libraries

In [1]:
import pandas as pd
import numpy as np

import sklearn
import optuna
import torch
import keras
import tensorflow as tf
import time
import os

from itertools import product

from pandas_profiling import ProfileReport

from chart_studio import plotly

import plotly.offline as pyoff
import plotly.graph_objs as go

from optuna.integration import PyTorchLightningPruningCallback
from optuna.visualization import plot_optimization_history, plot_contour, plot_param_importances

from pytorch_lightning.callbacks import EarlyStopping

from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, max_error, mean_absolute_percentage_error, r2_score

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

C:\Users\vinic\AppData\Local\Temp\ipykernel_9184\3664487231.py:14: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


# Functions

## Read file

In [2]:
def read_file(file):
    name_file, extension = os.path.splitext(file)

    if extension == ".csv":
        data = pd.read_csv(file)
    elif extension == ".xlsx":
        data = pd.read_excel(file)
    
    data = data.dropna()
    
    return data

## DF train and test

In [3]:
def df_train_test(data, target, test_size=0.3, random_state=42):
    sc = MinMaxScaler(feature_range=(0,1))

    x = sc.fit_transform(data.drop([target], axis=1))
    y = sc.fit_transform(data[target].values.reshape(-1,1))

    x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=test_size, random_state=random_state)

    return x_train, x_test, y_train, y_test

## Manual parameters

In [4]:
def create_fit_model(data, layer_config, optimizer, epochs=1000):
    model = Sequential()

    print("-"*50)
    print(f"layer_config: {layer_config}")
    print(f"optimizer: {optimizer}")
    print(f"epochs: {epochs}")

    for layer in layer_config:
        model.add(Dense(layer[0], activation=layer[1]))

    model.compile(loss="mean_squared_error", optimizer=optimizer)

    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, mode="min")

    x_train, x_test, y_train, y_test = data[0], data[1], data[2], data[3]
    start_time = time.time()
    history = model.fit(x_train, y_train, epochs=epochs, verbose=0, callbacks=[callback])
    y_pred = model.predict(x_test)
    stop_time = time.time()

    print("-"*50)
    print("Training time:", np.round((stop_time - start_time), 2),"s")
    print("-"*50)
    print(f"Accuracy score to {optimizer} & {layer}: {np.round(mean_squared_error(y_test, y_pred), 5)}")

## Grid Search

In [5]:
def grid_search(data, layer_config_list, optimizer_list, epochs_list):
    for layer_config, optimizer, epochs in product(layer_config_list, optimizer_list, epochs_list):
        print("-"*50)
        print(f"layer_config: {layer_config}")
        print(f"optimizer: {optimizer}")
        print(f"epochs: {epochs}")
        
        model = Sequential()
        for layer in layer_config:
            model.add(Dense(layer[0], activation=layer[1]))
        
        model.compile(loss="mean_squared_error", optimizer=optimizer)

        callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, mode="min")
        
        x_train, x_test, y_train, y_test = data[0], data[1], data[2], data[3]
        start_time = time.time()
        history = model.fit(x_train, y_train, epochs=epochs, verbose=0, callbacks=[callback])
        y_pred = model.predict(x_test)
        stop_time = time.time()

        print("-"*50)
        print("Training time:", np.round((stop_time - start_time), 2),"s")
        print("-"*50)
        print(f"Accuracy score: {np.round(mean_squared_error(y_test, y_pred), 5)}")
        print("\n"+"="*50+"\n")

# Testing

In [6]:
df = pd.read_excel(r"\Users\vinic\OneDrive\UFSCar\Dissertação\Codigo\data\Caso1_230kV_0.9_1.1.xlsx")
df.head()

,Ps1,Ps2,Ps3,Pd4,Pd5,Pd6,sigma
0,1.0,0.0,0.0,1.000000,0.0,0.000000,0.863621
1,1.0,0.0,0.0,0.846154,0.0,0.153846,0.848915
2,1.0,0.0,0.0,0.733333,0.0,0.266667,0.839086
3,1.0,0.0,0.0,0.647059,0.0,0.352941,0.832208
4,1.0,0.0,0.0,0.578947,0.0,0.421053,0.827203


In [7]:
# ProfileReport(df)

In [8]:
df_step_2 = df_train_test(df, target='sigma')


layer_config = [(5, "relu"), (1, "linear")]
optimizer = "Adam"
epochs = 1000

create_fit_model(df_step_2, layer_config, optimizer, epochs)

--------------------------------------------------
layer_config: [(5, 'relu'), (1, 'linear')]
optimizer: Adam
epochs: 1000
112/112 [==============================] - 0s 1ms/step
--------------------------------------------------
Training time: 9.23 s
--------------------------------------------------
Accuracy score to Adam & (1, 'linear'): 0.00937


In [9]:
layer_config_list = [
    [(5, "relu"), (1, "linear")],
    [(10, "relu"), (1, "linear")]
]

optimizer_list = ['Adam', 'SGD']

epochs_list = [100, 500]

grid_search(df_step_2, layer_config_list, optimizer_list, epochs_list)

--------------------------------------------------
layer_config: [(5, 'relu'), (1, 'linear')]
optimizer: Adam
epochs: 100
112/112 [==============================] - 0s 852us/step
--------------------------------------------------
Training time: 11.85 s
--------------------------------------------------
Accuracy score: 0.00172


--------------------------------------------------
layer_config: [(5, 'relu'), (1, 'linear')]
optimizer: Adam
epochs: 500
112/112 [==============================] - 0s 875us/step
--------------------------------------------------
Training time: 11.29 s
--------------------------------------------------
Accuracy score: 0.00174


--------------------------------------------------
layer_config: [(5, 'relu'), (1, 'linear')]
optimizer: SGD
epochs: 100
112/112 [==============================] - 0s 893us/step
--------------------------------------------------
Training time: 21.85 s
--------------------------------------------------
Accuracy score: 0.00745


-----------

# Optuna

In [10]:
results = []
def objective(trial):

    # Defina o número de unidades ocultas na camada oculta.
    num_hidden_units = trial.suggest_int('num_hidden_units', 5, 10, log=True)
    activation = trial.suggest_categorical('activation', ['relu', 'sigmoid', 'tanh'])
    optimizer = trial.suggest_categorical('optimizer', ['Adam', 'SGD'])

    # Defina a arquitetura da rede neural.
    model = Sequential()
    model.add(Dense(num_hidden_units, activation=activation))
    model.add(Dense(1, activation='linear'))

    # Compile o modelo.
    model.compile(loss="mean_squared_error", optimizer=optimizer)

    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, mode="min")

    # Treine o modelo.
    model.fit(x_train, y_train, epochs=100, verbose=0, callbacks=[callback])

    # Calcule o erro no conjunto de teste e retorne-a como a métrica a ser otimizada.
    y_pred = model.predict(x_test)
    mse = np.round(mean_squared_error(y_test, y_pred), 7)

    results.append({'num_hidden_units': num_hidden_units, 'activation': activation, 'optimizer': optimizer, 'value': mse})
    
    return mse

In [11]:
x_train, x_test, y_train, y_test = df_step_2[0], df_step_2[1], df_step_2[2], df_step_2[3]

# Crie um estudo Optuna e execute a otimização.
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

# Imprima os resultados.
print('Best trial:')
trial = study.best_trial
print('  Value: {}'.format(trial.value))
print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

[I 2023-02-28 10:40:50,887] A new study created in memory with name: no-name-5097346a-eed7-46f8-a367-8f38defc672f


112/112 [==============================] - 0s 839us/step


[I 2023-02-28 10:40:58,745] Trial 0 finished with value: 0.0012241 and parameters: {'num_hidden_units': 7, 'activation': 'relu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.0012241.


112/112 [==============================] - 0s 841us/step


[I 2023-02-28 10:41:22,635] Trial 1 finished with value: 0.0012455 and parameters: {'num_hidden_units': 5, 'activation': 'tanh', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.0012241.


...

112/112 [==============================] - 0s 835us/step


[I 2023-02-28 10:42:06,791] Trial 3 finished with value: 0.0013231 and parameters: {'num_hidden_units': 7, 'activation': 'relu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.0012241.


112/112 [==============================] - 0s 901us/step


[I 2023-02-28 10:42:23,034] Trial 4 finished with value: 0.0013335 and parameters: {'num_hidden_units': 6, 'activation': 'tanh', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.0012241.


Best trial:
  Value: 0.0012241
  Params: 
    num_hidden_units: 7
    activation: relu
    optimizer: Adam


In [12]:
df = pd.DataFrame(results)
df.sort_values('value')

,num_hidden_units,activation,optimizer,value
0,7,relu,Adam,0.001224
1,5,tanh,Adam,0.001246
3,7,relu,SGD,0.001323
4,6,tanh,Adam,0.001334
2,8,relu,SGD,0.001590


In [13]:
fig = optuna.visualization.plot_parallel_coordinate(study)
fig.show()

In [14]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()